# Phylogenetic Inference using PNUTS

This notebook gives an example on how to do phylogenetic inference using the PNUTS algorithm described in Wahle (2021) ([bioRxiv Paper](https://doi.org/10.1101/2021.03.16.435623)).

First the `MCPhylo` package and the `Random` package are loaded.

In [1]:
using MCPhylo;
using Random;
Random.seed!(1234);

The next step is to load the data.

In [2]:
tree, data = make_tree_with_data("Example.nex");

┌ Warning: Keyword datatype not understood, will be ignored
└ @ MCPhylo /home/gjaeger/.julia/packages/MCPhylo/KWPlY/src/Parser/ParseNexus.jl:92
┌ Warning: Keyword interleave not understood, will be ignored
└ @ MCPhylo /home/gjaeger/.julia/packages/MCPhylo/KWPlY/src/Parser/ParseNexus.jl:92


The `tree` object contains a r random tree over the leaves specified in the nexus file. You can view a newick string representing the tree by calling the `newick` function on the object.

In [3]:
newick(tree)

"((Lang1:0.124792,((Lang4:0.199377,Lang2:0.439243)6:0.749194,Lang3:0.110084)7:0.609875)8:0.761916,Lang5:0.588872)9:1.0;"

The input data needs to be stored in a dictionary to make it accessible to the sampler.

In [4]:
data_dictionary = Dict{Symbol, Any}(
  :data => data
);

Define a model by specifing a prior distribution on the equilibrium frequencis, a Dirichlet prior in this case, and a prior on the phylogenetic tree. In this example the Compound Dirichlet distribution (Zhang, Rannala and Yang 2012. ([paper](https://doi.org/10.1093/sysbio/sys030))) is chosen.

The distribution associated with the data is the `PhyloDist`. It is a distribution whose likelihood function is calculated according to Felsensteins Pruning algorithm ([paper](https://doi.org/10.1007/BF01734359)).

The *Restriction Site Model* for character evolution and no rate variation accross sites is chosen.

In [5]:
model =  Model(
    data = Stochastic(3, (tree, eq_freq) ->  PhyloDist(tree, eq_freq, [1.0], [1.0], Restriction), false),
    eq_freq = Stochastic(1, () -> Dirichlet(2,1),true),
    tree = Stochastic(Node(), () -> TreeDistribution(CompoundDirichlet(1.0,1.0,0.100,1.0)), true)
     )

Object of type "Model"
-------------------------------------------------------------------------------
eq_freq:
Object of type "Stochastic{Vector{Float64}, var"#17#18", MCPhylo.NullUnivariateDistribution}"
Float64[]
-------------------------------------------------------------------------------
tree:
Object of type "Stochastic{GeneralNode{Float64, Int64}, var"#19#20", MCPhylo.NullUnivariateDistribution}"
Tree with root:
"no_name"
Length:
0.0
Height:
0.0
-------------------------------------------------------------------------------
data:
Object of type "Stochastic{Array{Float64, 3}, var"#15#16", MCPhylo.NullUnivariateDistribution}"
Array{Float64, 3}(undef, 0, 0, 0)


Select the PNUTS sampler for the phylogenetic tree and the Slice sampler for the equilibrium frequency.

In [6]:
scheme = [PNUTS(:tree, target=0.7, targetNNI=0.5),
          SliceSimplex(:eq_freq),
          ]
setsamplers!(model, scheme);

Set initial values.

In [10]:
inits = [ Dict{Symbol, Union{Any, Real}}(
    :tree => tree,
    :eq_freq=> rand(Dirichlet(2,1)),
    :data => data_dictionary[:data]
    ) for c in 1:3
    ];

Run the MCMC. The statusbar is suppressed via the `verbose` argument, to avoid a cluttering of the output.

In [11]:
sim = mcmc(model, data_dictionary, inits, 5000, burnin=2500,thin=5, chains=3, trees=true, verbose=false)

Object of type "ModelChains"

Iterations = 2505:5000
Thinning interval = 5
Chains = 1,2,3
Samples per chain = 500
Burnin = 2500
[14.949601972776064 0.6671009131962964 0.33289908680369273 -29.743266796779302; 14.340244455308731 0.6367699746752989 0.3632300253246905 -29.744567244873174; … ; 33.313209490013406 0.6448943423608767 0.35510565763911967 -29.725502085050035; 38.14733254356905 0.4540779012531029 0.5459220987468935 -33.369059037187434;;; 23.253647203809727 0.6232761711948698 0.3767238288051333 -29.802829336440105; 18.5436822608004 0.6056925549813419 0.3943074450186609 -29.931111102066485; … ; 33.042730165406496 0.6412835136854962 0.3587164863145209 -29.732073538931957; 21.497579303440663 0.5245167366704084 0.47548326332960866 -31.252205073683918;;; 15.396343674766246 0.6553692560226685 0.3446307439773433 -29.721377729645248; 12.873654302931634 0.5837373399641527 0.41626266003585854 -30.171089516707646; … ; 23.05932038051192 0.6221577623461848 0.3778422376538226 -29.80932483478289

In [12]:
gelmandiag(sim)

                   PSRF 97.5%
Tree_length[tree] 1.033 1.106
       eq_freq[1] 0.999 1.000
       eq_freq[2] 0.999 1.000
       likelihood 1.002 1.002

